In [6]:
import spiceypy as spice
import pvl
import os
import re
import subprocess
import ale
import io
import re

# $messenger = /usgs/cpkgs/isis3/data/messsenger/
# $base = /usgs/cpkgs/isis3/data/base/

# Grabs the kernels and appends them to independent lists
def check_kern(kern):
    kern = kern.replace("$messenger/", os.environ['messenger'])
    kern = kern.replace("$base/", os.environ['base'])    
    filename, file_extension = os.path.splitext(kern)
    if(file_extension == '.bc'):
        bc_kerns.append(kern)
    elif(file_extension == '.tls'):
        tls_kerns.append(kern)
    elif(file_extension == '.tf'):
        tf_kerns.append(kern)
    elif(file_extension == '.tsc'):
        tsc_kerns.append(kern)
    elif(file_extension == '.bsp'):
        spk_kerns.append(kern)
    elif(file_extension == '.ti'):
        ti_kerns.append(kern)
    elif(file_extension == '.tpc'):
        tpc_kerns.append(kern)
    else:
        print("'{}' is not a valid kernel".format(kern))
        return 0
    
def get_intervals(bc_filename):
    interval = subprocess.check_output(["ckbrief", bc_filename])
    intervals = []

    for line in interval.splitlines():
        line = str(line)
        if('Object:' in line):
            intervals.append(int(line.strip("''").split()[1]))
            
    return intervals

def light_travel(kernels, sclk_start, sclk_stop, UTC=False):
    merc_id = 199
    mess_id = -236
    
    furnished_kerns = spice.furnsh(kernels)
    
    mess_state, mess_lt = spice.spkez(mess_id, (sclk_start + sclk_stop)/2, 'J2000', 'LT+S', 0)
    merc_state, merc_lt = spice.spkez(merc_id, (sclk_start + sclk_stop)/2, 'J2000', 'LT+S', 0)
    
    lt_pad = max(abs(mess_lt), abs(merc_lt)) + 15
    
    # Eph Time
    padded_start_et = sclk_start - lt_pad
    padded_end_et = sclk_stop + lt_pad
    
    # SCLK Padded Time
    padded_start_sclk = spice.sce2s(mess_id, padded_start_et)
    padded_end_sclk = spice.sce2s(mess_id, padded_end_et)
    
    # UTC Padded Time 
    if(UTC):
        padded_start_utc = spice.et2utc(padded_start_et, 'c', 3)
        padded_end_utc = spice.et2utc(padded_end_et, 'c', 3)
        
        return padded_start_utc, padded_end_utc
    
    return padded_start_sclk, padded_end_sclk
    
def ckslicer(bc_kern, tsc_kern, tls_kern, sclk_start, sclk_stop, reference_id):

    ck_filename, ck_file_extension = os.path.splitext(bc_kerns[0]) 
    #     sclk_start, sclk_stop =scl 
    sclk_stop = spice.sce2s(spacecraft_id, spice.scs2e(spacecraft_id, sclk_start) + exp_dur)

    ckslicer_command = ["./ckslicer", 
                        '-LSK {}'.format(tls_kern), 
                        '-SCLK {}'.format(tsc_kern), 
                        '-INPUTCK {}'.format(ck_filename + ck_file_extension), 
                        '-OUTPUTCK {}'.format("~/Desktop/ck" +'_sliced' + ck_file_extension),
                        '-ID {}'.format(reference_id),
                        '-TIMETYPE {}'.format('SCLK'),
                        '-START {}'.format(sclk_start),
                        '-STOP {}'.format(sclk_stop)]
    
    return subprocess.call(ckslicer_command), ckslicer_command  

In [7]:
# !ckbrief

# Enviorment variable that will be used to grab isis3 data
os.environ['messenger'] = "/usgs/cpkgs/isis3/data/messenger/"
os.environ['base'] = "/usgs/cpkgs/isis3/data/base/"
os.environ['spk_home'] = "/usgs/cpkgs/isis3/data/messenger/kernels/tspk/"

# Cube image path
cube_lab = "/Users/tthatcher/Desktop/FORKS/data/EN1072174528M.cub"

# Loads cube to extract data
cube_pvl = pvl.load(cube_lab)

# The kernels that the cub uses
parsed_kerns = cube_pvl['IsisCube']['Kernels']

# Starting SCLK string and ExposureDuration
sclk_start = cube_pvl['IsisCube']['Instrument']['SpacecraftClockCount']
exp_dur = cube_pvl['IsisCube']['Instrument']['ExposureDuration'].value * 0.001

# The fields we care about in the cub
required_elems = ['LeapSecond', 'TargetAttitudeShape', 'TargetPosition', 'InstrumentPointing', 'Instrument', 
                 'SpacecraftClock', 'InstrumentPosition', 'InstrumentAddendum']

# Lists to be used to store kernel vars
kernel_list = []
bc_kerns = []
tls_kerns = []
tf_kerns = []
tsc_kerns = []
spk_kerns = []
ti_kerns = []
tpc_kerns = []

for kern in required_elems:
    kern = parsed_kerns[kern]
    if(isinstance(kern, list)):
        for kernel in kern:
            check_kern(kernel)
    else:
        check_kern(kern)

space_intervals = get_intervals(bc_kerns[0])
target_spk_file = os.path.join(os.environ['spk_home'], "de423s.bsp")
kernels = bc_kerns + tsc_kerns + tls_kerns + spk_kerns + ti_kerns + tf_kerns + tsc_kerns

# Spacecraft / Reference ID of CK kerns
spacecraft_id = int(str(space_intervals[0])[:-3])
reference_id = int(str(space_intervals[0]))

# Gets SCLK times
spice.furnsh(kernels)
sclk_start = spice.scs2e(spacecraft_id, sclk_start)
sclk_stop = sclk_start + exp_dur

# Gets light corrected SCLK times
padded_sclk_start, padded_sclk_stop = light_travel(kernels, sclk_start, sclk_stop)

call, cmd = ckslicer(bc_kerns[0], tsc_kerns[0], tls_kerns[0], padded_sclk_start, padded_sclk_stop, reference_id)

'Table' is not a valid kernel
'Table' is not a valid kernel
'Table' is not a valid kernel


In [8]:
# Remove comments from the ck file
# NOTE: '~' will not work, use full path
new_kern = '/Users/tthatcher/Desktop/ck' +'_sliced' + '.bc'
commnt_command = ['commnt', '-d {}'.format(str(new_kern))]
subprocess.call(commnt_command)

# Makes a new txt file for the only comments that should be stored in the new ck file
with open("temp_commnts.txt","w+") as f:
    first_line = "This CK is for testing with the image: {}".format(cube_lab)
    second_line = "\nThis CK was generated using the following command: {}".format(" ".join(cmd))
    f.write(first_line + second_line)

new_commnts_command = ["commnt", "-a {}".format(new_kern), "temp_commnts.txt"]
subprocess.call(new_commnts_command)

0

In [9]:
# Transfer the subsetted ck into a xfr file
toxfr_command = "toxfr", "{}".format(new_kern)
subprocess.call(toxfr_command)

1

In [11]:
baseSPK = '''LEAPSECONDS_KERNEL     = <Your LSK kernel>
      SPK_KERNEL             = <the SPK your are creating>
         SOURCE_SPK_KERNEL   = <the SPK your are extracting from>
            INCLUDE_COMMENTS = no
            BODIES              = <the ID code of either your spacecraft or target>
            BEGIN_TIME          = <the start time of your image as a UTC string>
            END_TIME            = <the stop time of your image as a UTC stirng>'''

f = io.StringIO(baseSPK)
line = f.readline()
while():
    f.write(line.replace("<Your LSK kernel>", tls_kerns[0]))
    line = f.readline()

f.seek(0)
f.readlines()

['LEAPSECONDS_KERNEL     = <Your LSK kernel>\n',
 '      SPK_KERNEL             = <the SPK your are creating>\n',
 '         SOURCE_SPK_KERNEL   = <the SPK your are extracting from>\n',
 '            INCLUDE_COMMENTS = no\n',
 '            BODIES              = <the ID code of either your spacecraft or target>\n',
 '            BEGIN_TIME          = <the start time of your image as a UTC string>\n',
 '            END_TIME            = <the stop time of your image as a UTC stirng>']

In [ ]:
# !commnt -d ~/Desktop/ck_sliced.bc
# !./ckslicer -LSK /usgs/cpkgs/isis3/data/base/kernels/lsk/naif0012.tls -SCLK /usgs/cpkgs/isis3/data/messenger/kernels/sclk/messenger_2548.tsc -INPUTCK /usgs/cpkgs/isis3/data/messenger/kernels/ck/msgr_1504_v01.bc -OUTPUTCK ~/Desktop/ck_sliced.bc -ID -236001 -TIMETYPE SCLK -START "2/0072174528:989000" -STOP "2/072174528:990000"

In [ ]:
print(cube_pvl)

In [12]:
spaccraft_name = cube_pvl['IsisCube']['Instrument']['SpacecraftName']
target_name = cube_pvl['IsisCube']['Instrument']['TargetName']

# Convert strings to int ID's
craft_id = spice.bods2c(spaccraft_name)
target_id = spice.bods2c(target_name)
print(target_id)

199


In [13]:
def check_done(id_dict, base_id):
    for key in id_dict.keys():
        if(key != base_id):
            if(0 in id_dict[key]):
                continue
            else:
                return False, id_dict[key]
    return True, id_dict
                
def search_string(spk_file, body_id, id_dict=None):
    x = subprocess.check_output(["brief", "-c {}".format(spk_file)])
    
    # Convert from bytes
    x = x.decode("utf-8")
    f = io.StringIO(x)
    line = f.readline()
    
    if id_dict:
        id_dict[body_id] = set()
    else:
        id_dict = {body_id: set()}
        
    while(line):
        body_result = re.search(r'Body:(.*?)\n', line)
        
        if body_result:
            body_result = body_result.group(0)
            body_result = re.findall('\((.*?)\)', body_result)
            
            if(int(body_result[0]) == body_id):
                body_1 = int(body_result[0])
                body_2 = int(body_result[1])
                id_dict[body_1].add(body_2)
            
        else:
            if("w.r.t" in line):
                bodies_results = re.findall('\((.*?)\)', line)
                for key in id_dict.keys():
                    bodies_1 = int(bodies_results[0])
                    bodies_2 = int(bodies_results[1])
                    if(bodies_1 == key):
                        id_dict[key].add(bodies_2)
                        
        line = f.readline()
    
    return id_dict

def get_spk_ids(file):
    base_id = -236
    ids = search_string(file, base_id)
    
    # Search that a 0 is in the dict and return False otherwise
    for val in ids[base_id]:
        ids = search_string(file, val, id_dict=ids)
        status, vals = check_done(ids, base_id)
        if(status == True):
            continue
        else:
            # For each false value search again
            for unfinished_val in vals:
                search_string(file, unfinished_val, id_dict=ids)
    return ids

file = "/usgs/cpkgs/isis3/data/messenger/kernels/spk/msgr_20040803_20150430_od431sc_2.bsp"
ids = get_spk_ids(file)
print(ids)

{-236: {1, 2, 10, 399}, 1: {0}, 2: {0}, 10: {0}, 399: {3}, 3: {0}}


In [17]:
x = subprocess.check_output(["brief", "-c {}".format(file)])
x = x.decode("utf-8")
f = io.StringIO(x)
f.readlines()

[' \n',
 'BRIEF -- Version 4.0.0, September 8, 2010 -- Toolkit Version N0066\n',
 ' \n',
 ' \n',
 'Summary for: /usgs/cpkgs/isis3/data/messenger/kernels/spk/msgr_20040803_20150430_od431sc_2.bsp\n',
 ' \n',
 'Body: MESSENGER (-236) w.r.t. MERCURY BARYCENTER (1)\n',
 '      Start of Interval (ET)              End of Interval (ET)\n',
 '      -----------------------------       -----------------------------\n',
 '      2008 JAN 13 19:18:06.919            2008 JAN 15 18:52:08.364\n',
 '      2008 OCT 05 06:12:17.599            2008 OCT 07 11:08:05.670\n',
 '      2009 SEP 28 05:25:44.180            2009 OCT 01 14:07:55.870\n',
 '      2011 MAR 15 11:20:36.100            2015 APR 30 19:27:09.351\n',
 ' \n',
 'Body: MESSENGER (-236) w.r.t. VENUS BARYCENTER (2)\n',
 '      Start of Interval (ET)              End of Interval (ET)\n',
 '      -----------------------------       -----------------------------\n',
 '      2006 OCT 16 19:26:46.293            2006 OCT 31 22:15:29.222\n',
 '      200